# APPLICATION 11 - RNN TEXT GENERATOR

In [1]:
import torch
import torch.nn as nn
from torch.nn.utils import clip_grad_norm_
import os
import numpy as np

In [2]:
class Dictionary(object):
    def __init__(self):
        self.word2inx = {}
        self.idx2word = {}
        self.idx      = 0

    def add_word(self, word):
        if word not in self.word2inx:
            self.word2inx[word]     = self.idx
            self.idx2word[self.idx] = word
            self.idx += 1

    def __len__(self):
        return len(self.word2inx)

In [3]:
class TextProcess(object):
    def __init__(self):
        self.dictionary = Dictionary()

    def get_data(self, path, batch_size = 20):
        with open(path, 'r') as f:
            tokens = 0
            for line in f:
                words = line.split() + ['<eos>']
                tokens += len(words)
                for word in words:
                    self.dictionary.add_word(word)
    
        rep_tensor = torch.LongTensor(tokens)
        index      = 0
        with open(path, 'r') as f:
            for line in f:
                words = line.split() + ['<eos>']
                for word in words:
                    rep_tensor[index] = self.dictionary.word2inx[word]
                    index += 1
        
        num_batches = rep_tensor.shape[0] // batch_size
        rep_tensor  = rep_tensor[:num_batches * batch_size]
        rep_tensor  = rep_tensor.view(batch_size, -1)
        return rep_tensor

In [4]:
embed_size      = 128
hidden_size     = 1024
num_layers      = 1
num_epochs      = 20
batch_size      = 20
timesteps       = 30
learning_rate   = 0.002

In [5]:
corpus = TextProcess()
rep_tensor = corpus.get_data('data/alice.txt', batch_size)
print('Rep Tensor Shape: {}'.format(rep_tensor.shape))
vocab_size = len(corpus.dictionary)
print('Vocab Size: {}'.format(vocab_size))

num_batches = rep_tensor.shape[1] // timesteps
print('Num Batches: {}'.format(num_batches))

Rep Tensor Shape: torch.Size([20, 1484])
Vocab Size: 5290
Num Batches: 49


In [6]:
class TextGenerator(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size, num_layers):
        super(TextGenerator, self).__init__()
        self.embed  = nn.Embedding(vocab_size, embed_size)
        self.lstm   = nn.LSTM(embed_size, hidden_size, num_layers, batch_first = True)
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embed(x)
        out, (h, c) = self.lstm(x, h)
        out = out.reshape(out.size(0) * out.size(1), out.size(2))
        out = self.linear(out)
        return out, (h, c)

In [7]:
model = TextGenerator(vocab_size, embed_size, hidden_size, num_layers)

In [8]:
loss_fn   = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [9]:
for epoch in range(num_epochs):
    states = (torch.zeros(num_layers, batch_size, hidden_size),
              torch.zeros(num_layers, batch_size, hidden_size))

    for i in range(0, rep_tensor.size(1) - timesteps, timesteps):
        inputs  = rep_tensor[:, i     : i+timesteps]
        targets = rep_tensor[:, (i+1) : (i+1) +timesteps]

        outputs, _ = model(inputs, states)
        loss       = loss_fn(outputs, targets.reshape(-1))

        model.zero_grad()
        loss.backward()

        clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        step = (i+1) // timesteps
        if step % 100 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

Epoch [1/20], Loss: 8.5726
Epoch [2/20], Loss: 5.9708
Epoch [3/20], Loss: 5.1997
Epoch [4/20], Loss: 4.6462
Epoch [5/20], Loss: 4.1260
Epoch [6/20], Loss: 3.5226
Epoch [7/20], Loss: 3.0505
Epoch [8/20], Loss: 2.7080
Epoch [9/20], Loss: 2.2961
Epoch [10/20], Loss: 1.9707
Epoch [11/20], Loss: 1.6521
Epoch [12/20], Loss: 1.4473
Epoch [13/20], Loss: 1.1063
Epoch [14/20], Loss: 0.8074
Epoch [15/20], Loss: 0.6129
Epoch [16/20], Loss: 0.4348
Epoch [17/20], Loss: 0.2993
Epoch [18/20], Loss: 0.1703
Epoch [19/20], Loss: 0.1036
Epoch [20/20], Loss: 0.0789


In [11]:
with torch.no_grad():
    with open('data/result.txt', 'w') as f:
        state = (torch.zeros(num_layers, 1, hidden_size),
                 torch.zeros(num_layers, 1, hidden_size))
        
        input = torch.randint(0, vocab_size, (1,)).long().unsqueeze(1)

        for i in range(0, 500):
            output, _ = model(input, state)
            prob      = output.exp()
            word_id   = torch.multinomial(prob, num_samples = 1).item()

            input.fill_(word_id)

            word = corpus.dictionary.idx2word[word_id]
            word = '\n' if word == '<eos>' else word + ' '
            f.write(word)

            if (i+1) % 100 == 0:
                print('Sampled [{}/{}] words and to {}'.format(i+1, 500, 'data/result.txt'))

Sampled [100/500] words and to data/result.txt
Sampled [200/500] words and to data/result.txt
Sampled [300/500] words and to data/result.txt
Sampled [400/500] words and to data/result.txt
Sampled [500/500] words and to data/result.txt
